In [24]:
import os
import random
import warnings

import pandas as pd
import spacy
from dotenv import load_dotenv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sqlalchemy import create_engine

load_dotenv()
warnings.filterwarnings("ignore")

In [25]:
POSTGRES_ADDRESS = os.getenv("POSTGRES_ADDRESS")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_USERNAME = os.getenv("POSTGRES_USERNAME")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DBNAME = os.getenv("POSTGRES_DBNAME")

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

data = pd.read_sql_query("SELECT * FROM get_book_data", cnx)
data.fillna("missing", inplace=True)
data.shape

(58, 11)

In [26]:
list(data.columns)

['id',
 'book_title',
 'language',
 'publisher_name',
 'category_1',
 'category_2',
 'category_3',
 'author_1',
 'author_2',
 'short_description',
 'full_description']

In [27]:
nlp = spacy.blank("vi")
nlp.Defaults.stop_words.add("sách")


def clean_text(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


data["clean_text"] = (
        data["book_title"] + " " +
        data["publisher_name"] + " " +
        data["category_1"] + " " +
        data["category_2"] + " " +
        data["category_3"] + " " +
        data["author_1"] + " " +
        data["author_2"] + " " +
        data["short_description"] + " " +
        data["full_description"] + " "
)

data["clean_text"] = data["clean_text"].apply(clean_text)

data.iloc[random.choices(range(len(data)), k=10)]

,id,book_title,language,publisher_name,category_1,category_2,category_3,author_1,author_2,short_description,full_description,clean_text
41,104,Naruto - Tập 65,Tiếng việt,NXB Kim Đồng,Giả tưởng,Manga,missing,Masashi Kishimoto,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,naruto tập 65 nxb kim đồng giả tưởng manga mis...
28,91,Ông già và biển cả,Tiếng việt,NXB Trẻ,missing,missing,missing,Ernet Hemingway,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,ông già biển cả nxb trẻ missing missing missin...
7,68,Thám tử lừng danh Conan - Tập 19,Tiếng việt,NXB Kim Đồng,Manga,Trinh thám,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,thám tử lừng danh conan tập 19 nxb kim đồng ma...
24,87,Viên ngọc trai kì diệu,Tiếng việt,NXB Kim Đồng,Nước ngoài,Tiểu thuyết,missing,Elisa Sabatinelli,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,viên ngọc trai kì diệu nxb kim đồng nước ngoài...
16,79,Sách Giáo Khoa Lịch Sử Và Địa Lí Lớp 5,Tiếng việt,Giáo Dục Việt Nam,Sách giáo khoa,missing,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,giáo khoa lịch sử địa lí lớp 5 giáo dục việt n...
29,92,Alice ở xứ sở thần tiên,Tiếng việt,Dân Trí,Giả tưởng,Kinh điển,Nước ngoài,Lewis Carroll,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,alice xứ sở thần tiên dân trí giả tưởng kinh đ...
47,111,Tiểu thuyết Naruto - Itachi chân truyền - Ám d...,Tiếng việt,NXB Kim Đồng,Giả tưởng,Nước ngoài,Tiểu thuyết,Masashi Kishimoto,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,tiểu thuyết naruto itachi chân truyền ám thiên...
17,80,Sách Giáo Khoa Công Nghệ Lớp 5,Tiếng việt,Giáo Dục Việt Nam,Sách giáo khoa,missing,missing,missing,missing,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,giáo khoa công nghệ lớp 5 giáo dục việt nam ...
53,117,Love on the Brain,Tiếng anh,NXB Tổng Hợp TPHCM,Lãng mạn,Nước ngoài,Tiểu thuyết,Ali Hazelwood,missing,,,love on the brain nxb tổng hợp tphcm lãng mạ...
1,62,One punch man - Tập 18,Tiếng việt,NXB Kim Đồng,Giả tưởng,Manga,missing,One,Yusuke Murata,<p>Dolor labore incididunt eiusmod magna labor...,<h1>Sed elit dolor aliqua tempor lorem sit sit...,one punch man tập 18 nxb kim đồng giả tưởng ma...


In [28]:
cv = CountVectorizer()
clean_text_vector = cv.fit_transform(data['clean_text']).toarray()
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(clean_text_vector)

NearestNeighbors(algorithm='brute', metric='cosine')

In [29]:
def recommend(text, k=5):
    cleaned_text = cv.transform([clean_text(text)]).toarray()
    distances, indices = model.kneighbors(cleaned_text, n_neighbors=k)
    # return data.iloc[indices[0]][["id", "book_title"]].to_dict(orient="records")
    return data[["book_title", "publisher_name", "category_1", "category_2", "category_3", "author_1", "author_2"]].iloc[indices[0]]


In [34]:
user_description = "truyện manga dragon ball"
recommend(user_description)

,book_title,publisher_name,category_1,category_2,category_3,author_1,author_2
46,Dragon Ball - Tập 29,NXB Kim Đồng,Giả tưởng,Manga,missing,missing,missing
42,Dragon Ball - Tập 25,NXB Kim Đồng,Giả tưởng,Manga,missing,missing,missing
45,Dragon Ball - Tập 38,NXB Kim Đồng,Giả tưởng,Manga,missing,missing,missing
43,Dragon Ball - Tập 19,NXB Kim Đồng,Giả tưởng,Manga,Nước ngoài,missing,missing
44,Dragon Ball - Tập 35,NXB Kim Đồng,Giả tưởng,Manga,missing,missing,missing
